In [4]:
#importing library
import pandas as pd
import numpy as np
import plotly.express as px
from fbprophet import Prophet

In [12]:
#loading the data
df=pd.read_csv('../input/temperature-change/Environment_Temperature_change_E_All_Data_NOFLAG.csv',encoding='latin-1')
df.head()

,Area Code,Area,Months Code,Months,Element Code,Element,Unit,Y1961,Y1962,Y1963,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,2,Afghanistan,7001,January,7271,Temperature change,°C,0.777,0.062,2.744,...,3.601,1.179,-0.583,1.233,1.755,1.943,3.416,1.201,1.996,2.951
1,2,Afghanistan,7001,January,6078,Standard Deviation,°C,1.950,1.950,1.950,...,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950,1.950
2,2,Afghanistan,7002,February,7271,Temperature change,°C,-1.743,2.465,3.919,...,1.212,0.321,-3.201,1.494,-3.187,2.699,2.251,-0.323,2.705,0.086
3,2,Afghanistan,7002,February,6078,Standard Deviation,°C,2.597,2.597,2.597,...,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597,2.597
4,2,Afghanistan,7003,March,7271,Temperature change,°C,0.516,1.336,0.403,...,3.390,0.748,-0.527,2.246,-0.076,-0.497,2.296,0.834,4.418,0.234


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9656 entries, 0 to 9655
Data columns (total 66 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Area Code     9656 non-null   int64  
 1   Area          9656 non-null   object 
 2   Months Code   9656 non-null   int64  
 3   Months        9656 non-null   object 
 4   Element Code  9656 non-null   int64  
 5   Element       9656 non-null   object 
 6   Unit          9656 non-null   object 
 7   Y1961         8287 non-null   float64
 8   Y1962         8322 non-null   float64
 9   Y1963         8294 non-null   float64
 10  Y1964         8252 non-null   float64
 11  Y1965         8281 non-null   float64
 12  Y1966         8364 non-null   float64
 13  Y1967         8347 non-null   float64
 14  Y1968         8345 non-null   float64
 15  Y1969         8326 non-null   float64
 16  Y1970         8308 non-null   float64
 17  Y1971         8303 non-null   float64
 18  Y1972         8323 non-null 

In [32]:
#Preprocessing
def preprocess_inputs(df):
    df=df.copy()
    #selecting by the temperature change 
    df=df.query("Element=='Temperature change'")
    df=df.groupby('Area').mean()
    #only time series data
    df=df.loc[:,'Y1961':]
    # get global mean temperature change
    df=pd.DataFrame(df.mean()).reset_index(drop=False)
    df.columns=['ds','y']
    #changing the datatype
    df['ds']=df['ds'].apply(lambda x:x[1:]).astype(np.int)
    return df
    #

In [33]:
time_series=preprocess_inputs(df)

In [34]:
time_series

,ds,y
0,1961,0.143032
1,1962,-0.028398
2,1963,-0.026297
3,1964,-0.122865
4,1965,-0.224154
5,1966,0.095070
6,1967,-0.131975
7,1968,-0.167841
8,1969,0.105694
9,1970,0.072189


In [37]:
fig=px.line(time_series,
           x='ds',
           y='y',
           labels={'ds':'Year','y':'Change in temperature(C)'},
           title='Average Global Temperature Change Over time')
fig.show()

In-Sample Forecast

time_train=time_series.iloc[:44,:].copy()
time_test=time_series.iloc[44:,:].copy()

In [38]:
time_train=time_series.iloc[:44,:].copy() 
time_test=time_series.iloc[44:,:].copy()

In [40]:
time_test

,ds,y
44,2005,0.886901
45,2006,0.910877
46,2007,1.004826
47,2008,0.813313
48,2009,0.943937
49,2010,1.080097
50,2011,0.863045
51,2012,0.901637
52,2013,0.977131
53,2014,1.131417


In [41]:
in_model=Prophet()
in_model.fit(time_train)

Initial log joint probability = -56.4557
Iteration  1. Log joint probability =    21.8643. Improved by 78.32.
Iteration  2. Log joint probability =    46.5545. Improved by 24.6902.
Iteration  3. Log joint probability =    56.8179. Improved by 10.2634.
Iteration  4. Log joint probability =      56.91. Improved by 0.0921549.
Iteration  5. Log joint probability =    56.9158. Improved by 0.00579023.
Iteration  6. Log joint probability =     56.916. Improved by 0.000118252.
Iteration  7. Log joint probability =    56.9873. Improved by 0.071333.
Iteration  8. Log joint probability =     57.311. Improved by 0.323696.
Iteration  9. Log joint probability =    57.4718. Improved by 0.160785.
Iteration 10. Log joint probability =     57.483. Improved by 0.0112772.
Iteration 11. Log joint probability =    57.4903. Improved by 0.00725538.
Iteration 12. Log joint probability =    57.5646. Improved by 0.0742944.
Iteration 13. Log joint probability =    57.6696. Improved by 0.10499.
Iteration 14. Log j

In [46]:
in_forcast=in_model.predict(time_test).loc[:,['ds','yhat']]
in_forcast['ds']=in_forcast['ds'].apply(lambda x:x.year)
in_forcast

,ds,yhat
0,2005,0.811445
1,2006,0.809412
2,2007,0.796941
3,2008,0.774077
4,2009,0.901351
5,2010,0.899318
6,2011,0.886848
7,2012,0.863983
8,2013,0.991258
9,2014,0.989224


In [48]:
in_result=time_series.merge(in_forcast,on='ds',how='left')
in_result

,ds,y,yhat
0,1961,0.143032,NaN
1,1962,-0.028398,NaN
2,1963,-0.026297,NaN
3,1964,-0.122865,NaN
4,1965,-0.224154,NaN
5,1966,0.095070,NaN
6,1967,-0.131975,NaN
7,1968,-0.167841,NaN
8,1969,0.105694,NaN
9,1970,0.072189,NaN


In [53]:
fig=px.line(
in_result,
x='ds',
y=['y','yhat'],
color_discrete_sequence=['black','red'],
labels={'ds':'Year'},
title='Average Global Temperature change over time')
fig.show()

Out-OF-Sample Forecast


In [56]:
future_df=pd.DataFrame(np.arange(2020,2045),columns=['ds'])
future_df

,ds
0,2020
1,2021
2,2022
3,2023
4,2024
5,2025
6,2026
7,2027
8,2028
9,2029


In [57]:
out_model=Prophet()
out_model.fit(time_series)


Initial log joint probability = -70.3147
Iteration  1. Log joint probability =    17.5789. Improved by 87.8936.
Iteration  2. Log joint probability =    79.6322. Improved by 62.0532.
Iteration  3. Log joint probability =    101.037. Improved by 21.4051.
Iteration  4. Log joint probability =    101.972. Improved by 0.934885.
Iteration  5. Log joint probability =    102.012. Improved by 0.0398708.
Iteration  6. Log joint probability =    102.036. Improved by 0.024397.
Iteration  7. Log joint probability =    102.038. Improved by 0.00167987.
Iteration  8. Log joint probability =     102.04. Improved by 0.00156313.
Iteration  9. Log joint probability =    102.064. Improved by 0.0239864.
Iteration 10. Log joint probability =    102.114. Improved by 0.0506617.
Iteration 11. Log joint probability =    102.117. Improved by 0.00245446.
Iteration 12. Log joint probability =    102.133. Improved by 0.0161602.
Iteration 13. Log joint probability =    102.157. Improved by 0.0240189.
Iteration 14. L

In [66]:
out_forecast=out_model.predict(future_df).loc[:,['ds','yhat']]
out_forecast['ds']=out_forecast['ds'].apply(lambda x: x.year)
out_forecast

,ds,yhat
0,2020,1.216573
1,2021,1.328136
2,2022,1.354637
3,2023,1.354043
4,2024,1.326387
5,2025,1.437950
6,2026,1.464451
7,2027,1.463857
8,2028,1.436201
9,2029,1.547764


In [65]:
out_forecast

,ds,yhat
0,2020-01-01,1.216573
1,2021-01-01,1.328136
2,2022-01-01,1.354637
3,2023-01-01,1.354043
4,2024-01-01,1.326387
5,2025-01-01,1.437950
6,2026-01-01,1.464451
7,2027-01-01,1.463857
8,2028-01-01,1.436201
9,2029-01-01,1.547764


In [68]:
out_result_df=pd.concat([time_series,out_forecast],axis=0)
out_result_df

,ds,y,yhat
0,1961,0.143032,NaN
1,1962,-0.028398,NaN
2,1963,-0.026297,NaN
3,1964,-0.122865,NaN
4,1965,-0.224154,NaN
...,...,...,...
20,2040,NaN,1.765643
21,2041,NaN,1.877206
22,2042,NaN,1.903708
23,2043,NaN,1.903114


In [69]:
fig=px.line(
out_result_df,
x='ds',
y=['y','yhat'],
color_discrete_sequence=['black','red'],
labels={'ds':'Year'},
title='Average Global Temperature change over time')
fig.show()